# Coursera Capstone Project - The Battle of Neighbourhoods


### This Project is about finding a suitable neighbourhood in the City of Toronto to open a Restaurant chain

## Introduction

#### A group of 5 investors are plannning to open a restaurant chain in the city of Toronto. So they are interested to know which neighbourhood has most restaurants. And also to find the Neighbourhoods that are similar to the one which has most restaurants so as to be easy to decide upon the location.

## Data 
#### The location of each neighbourhood is scraped from wikipedia.The Borough name, neighbourhood name are taken from wikipedia page.  <br>                            The latitude and longitude of each neighbourhood is ob                                                                                                                                         Foursquare API is used to get the location of restaurants in the city of Toronto. And also to get the nearby top venues in each neighbourhood.

## Methodology

#### 1.Get the location of each Borough and Neighbourhood in Toronto using wikipedia page.<br>                                                                                                      2.Find the number of Neighbourhoods in each Borough. <br>                                                                                                                                                                   3.Get the restaurants locations and visualize it on map.<br>                                                                                                                                                                      4.Select the Boroughs and Neighbourhoods that are nearby to the one with most restaurants by checking out the number of neghbourhoods and visualizing      each     neighbourhood on map.<br>                                                                                                                                                                                                                 5.Use Kmeans Clustering algorithm to cluster the neighbourhoods. And select the ones that are nearby to the popular area with most restaurants and the ones that are similar to it.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

In [3]:
import pandas as pd
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df=pd.read_html(url, header=0)[0]

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Clean the data 

In [4]:
DF = df[df.Borough != 'Not assigned']
DF = df[df.Neighbourhood != 'Not assigned']
DF = DF[["Borough","Neighbourhood"]]
toronto_data = DF.reset_index(drop = True)
toronto_data.head()

,Borough,Neighbourhood
0,North York,Parkwoods
1,North York,Victoria Village
2,Downtown Toronto,Harbourfront
3,Downtown Toronto,Regent Park
4,North York,Lawrence Heights


In [6]:
Latitude = []
Longitude = []
for neigh,bou,i in zip(toronto_data['Neighbourhood'],toronto_data['Borough'],toronto_data.index):  
    address = "'"+neigh+ ","+ " "+ bou+"'"
    geolocator = Nominatim(user_agent="toronto_explorer")
    location = geolocator.geocode(address)
    if location != None:
        Latitude.append(location.latitude)
        Longitude.append(location.longitude)
        
    else:
        Latitude.append(0)
        Longitude.append(0)
        
print(Latitude)
print(Longitude)

[43.7612239, 43.732658, 0, 0, 43.7227784, 43.7220788, 43.7184123, 43.8049304, 43.8091955, 43.737178, 43.7120785, 0, 43.65815275, 43.6622763, 43.7087117, 43.633637, 43.6453376, 43.67145915, 43.67885645, 43.6631995, 43.7901172, 43.7802711, 43.7755039, 43.7184315, 43.737178, 43.6999302, 0, 0, 43.6353169, 43.6622732, 43.63123795, 43.6393732, 43.7548985, 43.7826012, 43.7689144, 43.6710244, 0, 0, 43.7598243, 43.7047983, 43.6603776, 43.6499915, 43.75646655, 43.7996637, 43.76389295, 43.7492988, 43.7405195, 43.704553, 43.65330575, 43.6576304, 43.6525334, 43.6653069, 43.6602019, 43.7437422, 43.7708175, 43.7695089, 43.7649819, 43.7541351, 43.7740965, 43.699971, 0, 0, 43.6563303, 43.64741325, 43.65467605, 43.7141672, 43.7359904, 43.724878, 43.7691966, 0, 43.7492988, 43.6863598, 43.6654704, 0, 0, 43.6509173, 43.63381345, 43.6371777, 43.7088231, 43.7278414, 43.6971738, 43.7612224, 43.7461836, 43.7492988, 43.6710244, 43.6678854, 0, 43.6541737, 43.7492988, 43.7186899, 43.7089606, 43.7600778, 43.721938

In [7]:
toronto_data['Latitude'] = Latitude
toronto_data['Longitude'] = Longitude
toronto_data = toronto_data[toronto_data.Latitude != 0].reset_index(drop = True)
toronto_data = toronto_data[toronto_data.Longitude != 0].reset_index(drop = True)
toronto_data

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.761224,-79.323986
1,North York,Victoria Village,43.732658,-79.311189
2,North York,Lawrence Heights,43.722778,-79.450933
3,North York,Lawrence Manor,43.722079,-79.437507
4,Etobicoke,Islington Avenue,43.718412,-79.556679
5,Scarborough,Rouge,43.804930,-79.165837
6,Scarborough,Malvern,43.809196,-79.221701
7,North York,Don Mills North,43.737178,-79.343451
8,East York,Woodbine Gardens,43.712078,-79.302567
9,Downtown Toronto,Ryerson,43.658153,-79.379080


### Now that the data is cleaned and ready, will proceed further.

### Lets get the Location of Toronto

In [11]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
Tlatitude = location.latitude
Tlongitude = location.longitude
print('Latitude =', Tlatitude)
print('Longitude =',Tlongitude)

Latitude = 43.653963
Longitude = -79.387207


### Now get the Restaurants location and visualize it on the Map.

#### Using the Foursquare API to get the location.

In [12]:
CLIENT_ID = 'MHUYK3THYV0W5VZXJOUBKELZMGS2OYCDOL5IKQBRGASOZP0F' # your Foursquare ID
CLIENT_SECRET = 'JYJVTEXPRGIONJVHLVW1SHMB1TYLJL5BRLRVC3CN2Y2L1U3R' # your Foursquare Secret
VERSION = '20190529'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MHUYK3THYV0W5VZXJOUBKELZMGS2OYCDOL5IKQBRGASOZP0F
CLIENT_SECRET:JYJVTEXPRGIONJVHLVW1SHMB1TYLJL5BRLRVC3CN2Y2L1U3R


In [27]:
search_query = 'restaurant'
radius = 50000
print(search_query + ' .... OK!')

restaurant .... OK!


In [28]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Tlatitude, Tlongitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=MHUYK3THYV0W5VZXJOUBKELZMGS2OYCDOL5IKQBRGASOZP0F&client_secret=JYJVTEXPRGIONJVHLVW1SHMB1TYLJL5BRLRVC3CN2Y2L1U3R&ll=43.653963,-79.387207&v=20190529&query=restaurant&radius=50000&limit=30'

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cf12c334c1f6753b03967af'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'primary': True,
      'shortName': 'American'}],
    'hasPerk': False,
    'id': '4ad4c05ff964a52048f720e3',
    'location': {'address': '110 Chestnut Street',
     'cc': 'CA',
     'city': 'Toronto',
     'country': 'Canada',
     'distance': 145,
     'formattedAddress': ['110 Chestnut Street',
      'Toronto ON M5G 1R3',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65488413420439,
       'lng': -79.38593077371578}],
     'lat': 43.65488413420439,
     'lng': -79.38593077371578,
     'postalCode': 'M5G 1R3',
     'state': 'ON'},
    'name': 'Hemispheres Restaurant & Bistro',
    'referralId': 'v-1559309363'},
   {'catego

In [30]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'name': 'American Restaurant', 'icon': {'suf...",False,4ad4c05ff964a52048f720e3,110 Chestnut Street,CA,Toronto,Canada,NaN,145,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'lat': 43.65488413420439, 'label': 'display'...",43.654884,-79.385931,NaN,M5G 1R3,ON,Hemispheres Restaurant & Bistro,v-1559309363,NaN
1,"[{'name': 'Dim Sum Restaurant', 'icon': {'suff...",False,4ad4c060f964a5207ff720e3,323 Spadina Ave.,CA,Toronto,Canada,at D'Arcy St.,922,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...","[{'lat': 43.65431754076345, 'label': 'display'...",43.654318,-79.398650,Kensington Market,M5T 2E9,ON,Rol San Restaurant 龍笙棧,v-1559309363,NaN
2,"[{'name': 'Restaurant', 'icon': {'suffix': '.p...",False,4ad4c05cf964a52006f620e3,37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,1011,[37 King Street East (at Le Meridien King Edwa...,"[{'lat': 43.64929834396347, 'label': 'display'...",43.649298,-79.376431,NaN,M5C 1E9,ON,Victoria's Restaurant,v-1559309363,498556908
3,"[{'name': 'Dim Sum Restaurant', 'icon': {'suff...",False,4b072e9df964a52009f922e3,280 Spadina Ave.,CA,Toronto,Canada,at Dundas St. W.,892,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...","[{'lat': 43.65278331265585, 'label': 'display'...",43.652783,-79.398174,NaN,NaN,ON,Sky Dragon Chinese Restaurant 龍翔酒樓,v-1559309363,NaN
4,"[{'name': 'Wine Bar', 'icon': {'suffix': '.png...",False,4ad4c05cf964a520dff520e3,301 Front St W,CA,Toronto,Canada,301 Front St. W,1271,"[301 Front St W (301 Front St. W), Toronto ON ...","[{'lat': 43.642537317144566, 'label': 'display...",43.642537,-79.387042,NaN,M5V 2T6,ON,360 Restaurant,v-1559309363,NaN


In [31]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,CA,Toronto,Canada,NaN,145,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'lat': 43.65488413420439, 'label': 'display'...",43.654884,-79.385931,NaN,M5G 1R3,ON,4ad4c05ff964a52048f720e3
1,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,CA,Toronto,Canada,at D'Arcy St.,922,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...","[{'lat': 43.65431754076345, 'label': 'display'...",43.654318,-79.398650,Kensington Market,M5T 2E9,ON,4ad4c060f964a5207ff720e3
2,Victoria's Restaurant,Restaurant,37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,1011,[37 King Street East (at Le Meridien King Edwa...,"[{'lat': 43.64929834396347, 'label': 'display'...",43.649298,-79.376431,NaN,M5C 1E9,ON,4ad4c05cf964a52006f620e3
3,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,CA,Toronto,Canada,at Dundas St. W.,892,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...","[{'lat': 43.65278331265585, 'label': 'display'...",43.652783,-79.398174,NaN,NaN,ON,4b072e9df964a52009f922e3
4,360 Restaurant,Wine Bar,301 Front St W,CA,Toronto,Canada,301 Front St. W,1271,"[301 Front St W (301 Front St. W), Toronto ON ...","[{'lat': 43.642537317144566, 'label': 'display...",43.642537,-79.387042,NaN,M5V 2T6,ON,4ad4c05cf964a520dff520e3


In [32]:
dataframe_filtered.name

0                    Hemispheres Restaurant & Bistro
1                             Rol San Restaurant 龍笙棧
2                              Victoria's Restaurant
3                 Sky Dragon Chinese Restaurant 龍翔酒樓
4                                     360 Restaurant
5                     Goldstone Noodle Restaurant 金石
6                             Azure Restaurant & Bar
7                      Green Tea Restaurant Downtown
8                            Swatow Restaurant 汕頭小食家
9                            Victor Restaurant & Bar
10                North-East Chinese Restaurant 華北美食
11    Sassafraz | Cafe | Restaurant | Private Events
12                     Hong Shing Chinese Restaurant
13                          New Sky Restaurant 小沙田食家
14                    The Hot House Restaurant & Bar
15                      Aroma Fine Indian Restaurant
16               Richtree Natural Market Restaurants
17                          Ka Chi Korean Restaurant
18                        Matisse Restaurant A

In [46]:
map_Neigh = folium.Map(location=[Tlatitude, Tlongitude], zoom_start=11)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='#000000',
        popup=label,
        fill = True,
        fill_color='#000000',
        fill_opacity=0.6
    ).add_to(map_Neigh)

map_Neigh

### Now visualize the neighbourhoods and select the ones nearby.

#### First find the number of Neighbourhoods in each Borough

In [47]:
toronto_data.groupby(['Borough']).count()

,Neighbourhood,Latitude,Longitude
Borough,,,
Central Toronto,15,15,15
Downtown Toronto,15,15,15
East Toronto,5,5,5
East York,5,5,5
Etobicoke,43,43,43
North York,37,37,37
Scarborough,37,37,37
West Toronto,12,12,12
York,7,7,7


### When we explore the above map, we find that the Borough - Central Toronto, Downtown Toronto, East Toronto, Etobicoke, York are the ones nearby to it. Hence we proceed only with those Boroughs.

In [49]:
CT_data = toronto_data[toronto_data['Borough'] == 'Central Toronto'].reset_index(drop=True)
CT_data

DT_data = toronto_data[toronto_data['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DT_data

ET_data = toronto_data[toronto_data['Borough'] == 'East Toronto'].reset_index(drop=True)
ET_data


etobicoke_data = toronto_data[toronto_data['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicoke_data

Y_data = toronto_data[toronto_data['Borough'] == 'York'].reset_index(drop=True)
Y_data

,Borough,Neighbourhood,Latitude,Longitude
0,York,Del Ray,38.837669,-77.049739
1,York,Keelesdale,43.690157,-79.475056
2,York,Mount Dennis,43.686960,-79.489551
3,York,Silverthorn,43.682123,-79.471572
4,York,The Junction North,42.928672,-78.026122
5,York,Runnymede,43.663524,-79.489459
6,York,Weston,43.689619,-79.479188


In [51]:
address = 'Central Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
CTlatitude = location.latitude
CTlongitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(CTlatitude, CTlongitude))


The geograpical coordinate of Central Toronto are 43.653963, -79.387207.


In [52]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
DTlatitude = location.latitude
DTlongitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(DTlatitude, DTlongitude))


The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


In [53]:
address = 'East Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
ETlatitude = location.latitude
ETlongitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(ETlatitude, ETlongitude))


The geograpical coordinate of Downtown Toronto are 43.653963, -79.387207.


In [50]:
address = 'Etobicoke, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
Elatitude = location.latitude
Elongitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(Elatitude, Elongitude))



The geograpical coordinate of Etobicoke are 43.67145915, -79.5524920661167.


In [54]:
address = 'York, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
Ylatitude = location.latitude
Ylongitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(Ylatitude, Ylongitude))



The geograpical coordinate of Etobicoke are 43.6896191, -79.479188.


In [55]:

# create map of Manhattan using latitude and longitude values
map_Neigh = folium.Map(location=[Tlatitude, Tlongitude], zoom_start=11)


folium.features.CircleMarker(
    [Elatitude, Elongitude],
    radius=10,
    color='red',
    popup='Etobicoke',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh)

# add markers to map
for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#418c3f',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)  
    
    
folium.features.CircleMarker(
    [Ylatitude, Ylongitude],
    radius=10,
    color='red',
    popup='York',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh) 

for lat, lng, label in zip(Y_data['Latitude'], Y_data['Longitude'], Y_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3f2199',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)  
    
folium.features.CircleMarker(
    [ETlatitude, ETlongitude],
    radius=10,
    color='red',
    popup='Scarborough',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh)     
    
    
for lat, lng, label in zip(ET_data['Latitude'], ET_data['Longitude'], ET_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#ecef32',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)     
    
map_Neigh

In [56]:



folium.features.CircleMarker(
    [CTlatitude, CTlongitude],
    radius=10,
    color='red',
    popup='Central Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh)

# add markers to map
for lat, lng, label in zip(CT_data['Latitude'], CT_data['Longitude'], CT_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#ea2ada',
        fill=True,
        fill_color='#ea2ada',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)  
    
    
folium.features.CircleMarker(
    [DTlatitude, DTlongitude],
    radius=10,
    color='red',
    popup='Downtown Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh) 

for lat, lng, label in zip(DT_data['Latitude'], DT_data['Longitude'], DT_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#0cf9f9',
        fill=True,
        fill_color='#0cf9f9',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)  
    
    
map_Neigh

In [60]:
NY_data = toronto_data[toronto_data['Borough'] == 'North York'].reset_index(drop=True)
NY_data

SC_data = toronto_data[toronto_data['Borough'] == 'Scarborough'].reset_index(drop=True)
SC_data

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,Rouge,43.804930,-79.165837
1,Scarborough,Malvern,43.809196,-79.221701
2,Scarborough,Highland Creek,43.790117,-79.173334
3,Scarborough,Rouge Hill,43.780271,-79.130499
4,Scarborough,Port Union,43.775504,-79.134976
5,Scarborough,Guildwood,43.754899,-79.197776
6,Scarborough,Morningside,43.782601,-79.204958
7,Scarborough,West Hill,43.768914,-79.187291
8,Scarborough,Woburn,43.759824,-79.225291
9,Scarborough,Cedarbrae,43.756467,-79.226692


### Let's include 2 more Boroughs - Scarborough and North York just to cover larger part of Toronto.

In [57]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
Slatitude = location.latitude
Slongitude = location.longitude
print('The geograpical coordinate of Scarborough, Toronto are {}, {}.'.format(Slatitude, Slongitude))

The geograpical coordinate of Scarborough, Toronto are 43.773077, -79.257774.


In [58]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
NYlatitude = location.latitude
NYlongitude = location.longitude
print('The geograpical coordinate of North York, Toronto are {}, {}.'.format(NYlatitude, NYlongitude))

The geograpical coordinate of North York, Toronto are 43.7708175, -79.4132998.


In [61]:
folium.features.CircleMarker(
    [NYlatitude, NYlongitude],
    radius=10,
    color='red',
    popup='North York',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh) 

for lat, lng, label in zip(NY_data['Latitude'], NY_data['Longitude'], NY_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#1a4a99',
        fill=True,
        fill_color='#1a4a99',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)  
    
folium.features.CircleMarker(
    [Slatitude, Slongitude],
    radius=10,
    color='red',
    popup='Scarborough',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh)     
    
    
for lat, lng, label in zip(SC_data['Latitude'], SC_data['Longitude'], SC_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#68ffd4',
        fill=True,
        fill_color='#68ffd4',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)     
    
map_Neigh

In [62]:

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='#000000',
        popup=label,
        fill = True,
        fill_color='#000000',
        fill_opacity=0.6
    ).add_to(map_Neigh)

map_Neigh

## We can see that  Neighbourhoods of Central Toronto and Downtown Toronto has most number of Restaurants. 


### Let's proceed with the clustering of Neighbourhoods to find the ones similar to the ones in Central Toronto and Downtown Toronto.

In [63]:
df = etobicoke_data.append(NY_data)
df = df.append(CT_data)
df = df.append(DT_data)
df = df.append(ET_data)
df = df.append(SC_data)

In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [65]:
# type your answer here

neigh_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )



Islington Avenue
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Westmount
Kingsview Village
Martin Grove Gardens
Richview Gardens
St. Phillips
Mimico South
New Toronto
Albion Gardens
Humbergate
Jamestown
Mount Olive
Silverstone
South Steeles
Thistletown
Alderwood
Long Branch
Northwest
The Kingsway
Montgomery Road
Old Mill North
Humber Bay
King's Mill Park
Kingsway Park South East
Mimico NE
Old Mill South
The Queensway East
Royal York South East
Sunnylea
Kingsway Park South West
Mimico NW
The Queensway West
Royal York South West
South of Bloor
Parkwoods
Victoria Village
Lawrence Heights
Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park
Don Mills South
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Fairview
Henry Farm
Oriole
Northwood Park
York University
Bayview Village
Downsview East
Silver Hills
York Mills
Downsview West
Downsview
North Park
Upwood Park
Humber Summit
Newtonbrook
Will

In [66]:
print(neigh_venues.shape)
neigh_venues.head()

(2327, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Islington Avenue,43.718412,-79.556679,Chubby's Fish N' Chips,43.715190,-79.555346,Fish & Chips Shop
1,Islington Avenue,43.718412,-79.556679,Subway,43.716150,-79.554667,Sandwich Place
2,Islington Avenue,43.718412,-79.556679,Popeyes Louisiana Kitchen,43.715939,-79.554628,Fried Chicken Joint
3,Islington Avenue,43.718412,-79.556679,Afghan Kebob Cuisine,43.715451,-79.555523,Afghan Restaurant
4,Islington Avenue,43.718412,-79.556679,Fresh Value,43.716153,-79.554566,Grocery Store


In [67]:
neigh_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,30,30,30,30,30,30
Agincourt,14,14,14,14,14,14
Agincourt North,30,30,30,30,30,30
Albion Gardens,12,12,12,12,12,12
Alderwood,9,9,9,9,9,9
Bathurst Manor,4,4,4,4,4,4
Bayview Village,11,11,11,11,11,11
Bedford Park,1,1,1,1,1,1
Birch Cliff,4,4,4,4,4,4


In [68]:
print('There are {} uniques categories.'.format(len(neigh_venues['Venue Category'].unique())))

There are 230 uniques categories.


In [69]:
# one hot encoding
neigh_onehot = pd.get_dummies(neigh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neigh_onehot['Neighborhood'] = neigh_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neigh_onehot.columns[-1]] + list(neigh_onehot.columns[:-1])
neigh_onehot = neigh_onehot[fixed_columns]

neigh_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Bar,Beer Store,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cantonese Restaurant,Caribbean Restaurant,Castle,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hong Kong Restaurant,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Light Rail Station,Liquor Store,Locksmith,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rock Club,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Islington Avenue,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [70]:
(neigh_onehot.shape)

(2327, 230)

In [71]:
neigh_grouped = neigh_onehot.groupby('Neighborhood').mean().reset_index()
neigh_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Bar,Beer Store,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cantonese Restaurant,Caribbean Restaurant,Castle,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hong Kong Restaurant,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Light Rail Station,Liquor Store,Locksmith,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rock Club,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [72]:
neigh_grouped.shape

(151, 230)

### Let's get top 5 venues from each Neighbourhood

In [73]:
num_top_venues = 5

for hood in neigh_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neigh_grouped[neigh_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                venue  freq
0         Coffee Shop  0.20
1  Italian Restaurant  0.10
2             Theater  0.07
3      Breakfast Spot  0.07
4      Farmers Market  0.03


----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.29
1   Rental Car Location  0.14
2  Cantonese Restaurant  0.07
3      Asian Restaurant  0.07
4     Korean Restaurant  0.07


----Agincourt North----
                venue  freq
0      Ice Cream Shop  0.07
1  Chinese Restaurant  0.07
2              Bakery  0.07
3   Convenience Store  0.03
4                 Spa  0.03


----Albion Gardens----
            venue  freq
0   Grocery Store  0.17
1     Coffee Shop  0.08
2  Sandwich Place  0.08
3      Beer Store  0.08
4  Hardware Store  0.08


----Alderwood----
         venue  freq
0  Pizza Place  0.22
1          Gym  0.11
2     Pharmacy  0.11
3          Pub  0.11
4  Coffee Shop  0.11


----Bathurst Manor----
               venue  freq
0         Playground  0.25
1  Convenience Store  0.25


In [74]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [94]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neigh_grouped['Neighborhood']

for ind in np.arange(neigh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neigh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Italian Restaurant,Theater,Breakfast Spot,Yoga Studio,Gastropub,Furniture / Home Store,Spa,Liquor Store,Mexican Restaurant
1,Agincourt,Chinese Restaurant,Rental Car Location,Train Station,Cantonese Restaurant,Korean Restaurant,Restaurant,Coffee Shop,Asian Restaurant,Hong Kong Restaurant,Vietnamese Restaurant
2,Agincourt North,Ice Cream Shop,Chinese Restaurant,Bakery,Clothing Store,Gift Shop,Movie Theater,Sandwich Place,Juice Bar,Coffee Shop,Sporting Goods Shop
3,Albion Gardens,Grocery Store,Pizza Place,Auto Garage,Pharmacy,Fried Chicken Joint,Beer Store,Fast Food Restaurant,Hardware Store,Japanese Restaurant,Sandwich Place
4,Alderwood,Pizza Place,Pharmacy,Gym,Skating Rink,Sandwich Place,Pool,Coffee Shop,Pub,Dumpling Restaurant,Eastern European Restaurant


In [90]:
# set number of clusters
kclusters = 21

neigh_grouped_clustering = neigh_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neigh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([ 6,  2,  2,  0,  9, 12,  9,  4,  3,  1], dtype=int32)

In [91]:
# add clustering labels
labels = kmeans.labels_.astype(int)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels3', labels)

neigh_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neigh_merged = neigh_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

neigh_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels3,Cluster Labels2,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue
0,Etobicoke,Islington Avenue,43.718412,-79.556679,2.0,29.0,2.0,0.0,Grocery Store,Video Game Store,Sandwich Place,Bookstore,Restaurant,Fish & Chips Shop,Arts & Crafts Store,Pet Store,Fried Chicken Joint,African Restaurant,Afghan Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Fish Market,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Women's Store,Eastern European Restaurant
1,Etobicoke,Cloverdale,43.633637,-79.549745,9.0,3.0,9.0,0.0,Vietnamese Restaurant,Coffee Shop,Fast Food Restaurant,Liquor Store,Bakery,Shopping Mall,Sandwich Place,Café,Restaurant,Rental Car Location,Beer Store,Furniture / Home Store,Supermarket,Sushi Restaurant,Record Shop,Pub,Clothing Store,Pizza Place,Discount Store,Pharmacy,Pet Store
2,Etobicoke,Islington,43.645338,-79.524870,6.0,15.0,6.0,0.0,Coffee Shop,Pizza Place,Bus Stop,Sandwich Place,Sushi Restaurant,Pub,Thai Restaurant,Fast Food Restaurant,Restaurant,Caribbean Restaurant,Beer Store,Korean Restaurant,Eastern European Restaurant,Bakery,Flower Shop,BBQ Joint,Convenience Store,Japanese Restaurant,Gym,Thrift / Vintage Store,Fish & Chips Shop
3,Etobicoke,Martin Grove,43.671459,-79.552492,5.0,6.0,5.0,3.0,Playground,Supermarket,Coffee Shop,Flower Shop,Garden,Women's Store,Historic Site,Home Service,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dongbei Restaurant
4,Etobicoke,Princess Gardens,43.678856,-79.556456,1.0,47.0,1.0,3.0,Pharmacy,Liquor Store,Supermarket,Coffee Shop,Women's Store,Electronics Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Empanada Restaurant,Eastern European Restaurant,Food Truck,Dumpling Restaurant,Drugstore,Donut Shop


In [92]:
neigh_merged = neigh_merged[~neigh_merged['Cluster Labels3'].isnull()]


# final conversion now worked
neigh_merged[['Cluster Labels3']] = neigh_merged[['Cluster Labels3']].astype(int)

In [95]:
# create map
map_clusters = folium.Map(location=[Tlatitude, Tlongitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neigh_merged['Latitude'], neigh_merged['Longitude'], neigh_merged['Neighbourhood'], neigh_merged['Cluster Labels3']):
    if np.isnan(cluster):
        cluster = np.nan_to_num(cluster)
    
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
    

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='#000000',
        popup=label,
        fill = True,
        fill_color='#000000',
        fill_opacity=0.6
    ).add_to(map_clusters)

    
       
map_clusters

In [85]:
neigh_merged.loc[neigh_merged['Cluster Labels1'] == 0, neigh_merged.columns[[1] + list(range(5, neigh_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue
17,Albion Gardens,0.0,Grocery Store,Pizza Place,Auto Garage,Pharmacy,Fried Chicken Joint,Beer Store,Fast Food Restaurant,Hardware Store,Japanese Restaurant,Sandwich Place,Coffee Shop,Empanada Restaurant,Farmers Market,Event Space,Falafel Restaurant,Eastern European Restaurant,Filipino Restaurant,Fish & Chips Shop,Electronics Store,Donut Shop,Dumpling Restaurant
5,Glencairn,0.0,Playground,Pub,Grocery Store,Japanese Restaurant,Pizza Place,Dessert Shop,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Dumpling Restaurant,Design Studio,Drugstore,Food & Drink Shop
11,East Birchmount Park,10.0,Grocery Store,Intersection,Gas Station,Beer Store,Women's Store,Food Court,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,French Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore
13,Kennedy Park,8.0,Fast Food Restaurant,Asian Restaurant,Grocery Store,Women's Store,Electronics Store,Food Court,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Farmers Market,Falafel Restaurant,Event Space,Empanada Restaurant,Eastern European Restaurant,French Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dongbei Restaurant
14,Clairlea,8.0,Diner,Fast Food Restaurant,Grocery Store,Sandwich Place,Mexican Restaurant,Convenience Store,Dessert Shop,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Design Studio,Farmers Market,Falafel Restaurant,Event Space,Empanada Restaurant,Dim Sum Restaurant,Electronics Store,Eastern European Restaurant,Food Court


## Results


#### We can see that most of the restaurants are based on Central Toronto and Dowtown Toronto.
#### The Black circles represents Restaurant locations and the other circles represents the similar clustered Neighbourhoods.



## Conclusion

### Based on above results, we can recommend the following top 3 Neighbourhoods:
### 1.Adelaide in East Toronto<br>                                                                                                                                                                                                                                                 2.King and Spadina in Downtown Toronto<br>                                                                                                                                                                                                                                 3.South Hill in Central Toronto